# Polymarket Inventory MM Quoter

## 4-Layer Market Making Framework

This notebook implements a market-making strategy for **Polymarket binary UP/DOWN markets** 
(e.g., "Will BTC be above $97,000 at 2:00 PM?").

---

## The Core Problem

In Polymarket binary markets:
- Two complementary assets exist: **UP** and **DOWN**
- At resolution: `UP + DOWN = $1.00` (one pays $1, other pays $0)
- **Profit** = `$1.00 - (avg_cost_up + avg_cost_down)` when you hold both sides
- **Challenge**: Accumulate BOTH sides at a combined cost < $1.00
- **Risk**: Informed traders sell you the losing side at "fair" prices

---

## The 4-Layer Framework

```
┌─────────────────────────────────────────────────────────────────┐
│                         INPUTS                                   │
├─────────────────┬─────────────────┬─────────────────────────────┤
│ ORACLE          │ POLYMARKET      │ YOUR INVENTORY              │
│ (WebSocket)     │ (API/WS)        │ (Internal State)            │
├─────────────────┼─────────────────┼─────────────────────────────┤
│ BTC = $97,200   │ UP: 53/55c      │ UP: 100 @ 52c               │
│ Threshold=$97k  │ DOWN: 46/48c    │ DOWN: 80 @ 48c              │
│ (+0.2% above)   │                 │ Imbalance: +11%             │
└────────┬────────┴────────┬────────┴────────┬────────────────────┘
         │                 │                 │
         ▼                 ▼                 ▼
┌─────────────────────────────────────────────────────────────────┐
│                   4-LAYER PROCESSING                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  LAYER 1: ORACLE-ADJUSTED OFFSET                                │
│  ───────────────────────────────                                │
│  Adjust offset from best_ask based on oracle direction           │
│  oracle_adj = (97200 - 97000) / 97000 × 5.0 = +1.0c             │
│  UP offset = 2.2c - 1.0c = 1.2c (TIGHT - aggressive)            │
│  DOWN offset = 2.2c + 1.0c = 3.2c (WIDE - protection)           │
│                                                                  │
│  LAYER 2: ADVERSE SELECTION SPREAD                              │
│  ─────────────────────────────────                              │
│  Widen base spread near resolution (informed traders)            │
│  p_informed = 0.2 × exp(-minutes / 5) = ~4% at 8 min            │
│  base_spread = 0.02 × (1 + 3 × 0.04) = 2.2c                     │
│                                                                  │
│  LAYER 3: INVENTORY SKEW                                        │
│  ───────────────────────                                        │
│  Adjust quotes to balance inventory (affects PRICES and SIZES)   │
│  q = +11% (overweight UP)                                        │
│  UP spread_mult = 1 + 0.5 × 0.11 = 1.06 (wider offset)          │
│  DOWN spread_mult = 1 - 0.5 × 0.11 = 0.94 (tighter offset)      │
│  UP size = 50 × exp(-1 × 0.11) = 45 shares (smaller)            │
│  DOWN size = 50 × exp(+1 × 0.11) = 56 shares (bigger)           │
│                                                                  │
│  LAYER 4: EDGE CHECK                                            │
│  ───────────────────                                            │
│  Only quote if bid is meaningfully below market ask              │
│  edge = market_ask - our_bid                                    │
│  If edge < 1c threshold → DON'T QUOTE                           │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────────────────────────┐
│                         OUTPUT                                   │
├─────────────────────────────────────────────────────────────────┤
│  UP:   BID 54c for 45 shares (1c edge)                          │
│  DOWN: BID 45c for 56 shares (3c edge)                          │
│  Combined bid: 99c (1c profit per pair if both fill)            │
└─────────────────────────────────────────────────────────────────┘
```

---

## Why No Profitability Cap?

**Balance is the real safeguard:**
- In binary markets: UP + DOWN ≈ $1.00 at any time
- If you always quote BOTH sides at market prices, combined cost ≈ $1.00
- Static caps based on historical avg become stale when markets move
- Tight imbalance control ensures balanced fills

---

## Theoretical Foundation

Adapted from O'Hara's *Market Microstructure Theory*:
- **Chapter 2**: Inventory Models (balance both sides)
- **Chapter 3**: Glosten-Milgrom (adverse selection → widen spread for informed traders)

In [ ]:
# ==============================================================================
# IMPORTS AND CONSTANTS
# ==============================================================================

import math
from dataclasses import dataclass
from typing import Optional, Dict, Any

# ------------------------------------------------------------------------------
# POLYMARKET TICK SIZE CONSTRAINT
# ------------------------------------------------------------------------------
# Polymarket only accepts prices in whole cents (0.01, 0.02, ... 0.99)
# Any price like 0.515 or 0.4875 is INVALID and will be rejected
# All our calculations must snap to this tick size before placing orders

TICK_SIZE = 0.01  # 1 cent

def snap_to_tick(value: float) -> float:
    """
    Snap a price to the nearest valid Polymarket tick (whole cent).
    
    Examples:
        snap_to_tick(0.515)  -> 0.52
        snap_to_tick(0.494)  -> 0.49
        snap_to_tick(0.4875) -> 0.49
    
    This is CRITICAL - Polymarket will reject orders with sub-cent prices!
    """
    return round(round(value / TICK_SIZE) * TICK_SIZE, 2)

print("Tick size constraint: All prices must be whole cents (0.01 increments)")
print(f"Example: snap_to_tick(0.515) = {snap_to_tick(0.515)}")
print(f"Example: snap_to_tick(0.494) = {snap_to_tick(0.494)}")

---

## 1. Data Structures

We need three pieces of information to generate quotes:

1. **Inventory** - What positions do we currently hold? At what average prices?
2. **Market** - What are the current bid/ask prices on Polymarket?
3. **Oracle** - What is the current BTC/ETH price from the exchange?

Each is represented as a dataclass for clarity.

In [1]:
# ==============================================================================
# DATA STRUCTURE: INVENTORY
# ==============================================================================
# Tracks our current positions in UP and DOWN tokens
#
# KEY INSIGHT: In binary markets, profit comes from holding BOTH sides
# If you hold 100 UP @ 48c and 100 DOWN @ 48c:
#   - Combined cost = 48c + 48c = 96c per pair
#   - At resolution, one side pays $1, other pays $0
#   - But you have PAIRS, so you get $1 per pair
#   - Profit = $1.00 - $0.96 = 4c per pair = $4.00 total
# ==============================================================================

@dataclass
class Inventory:
    """
    Current position in UP and DOWN tokens.
    
    Attributes:
        up_qty: Number of UP tokens held
        up_avg: Average cost per UP token (e.g., 0.48 = 48 cents)
        down_qty: Number of DOWN tokens held
        down_avg: Average cost per DOWN token
    
    Example:
        inv = Inventory(up_qty=100, up_avg=0.48, down_qty=80, down_avg=0.50)
        # We hold 100 UP tokens bought at average 48c each
        # We hold 80 DOWN tokens bought at average 50c each
    """
    up_qty: float
    up_avg: float
    down_qty: float
    down_avg: float
    
    @property
    def combined_avg(self) -> float:
        """
        Total cost per pair = up_avg + down_avg
        
        This is THE KEY METRIC for profitability:
        - If combined_avg < 1.00: We profit on matched pairs
        - If combined_avg > 1.00: We lose on matched pairs (UNDERWATER)
        - If combined_avg = 1.00: Breakeven
        
        Example:
            up_avg=0.48, down_avg=0.50 → combined=0.98 → 2c profit per pair
        """
        return self.up_avg + self.down_avg
    
    @property
    def imbalance(self) -> float:
        """
        Normalized inventory imbalance: ranges from -1 to +1
        
        Formula: q = (UP_qty - DOWN_qty) / (UP_qty + DOWN_qty)
        
        Interpretation:
            q = +1.0: 100% UP, 0% DOWN (extreme overweight UP)
            q = +0.5: 75% UP, 25% DOWN (overweight UP)
            q =  0.0: 50% UP, 50% DOWN (perfectly balanced)
            q = -0.5: 25% UP, 75% DOWN (overweight DOWN)
            q = -1.0: 0% UP, 100% DOWN (extreme overweight DOWN)
        
        WHY IT MATTERS:
            - Unmatched inventory is RISKY - you're exposed to direction
            - If overweight UP and DOWN wins, unmatched UP tokens = $0
            - Goal is to stay balanced (q ≈ 0) so you can merge everything
        """
        total = self.up_qty + self.down_qty
        if total == 0:
            return 0.0
        return (self.up_qty - self.down_qty) / total
    
    @property
    def pairs(self) -> float:
        """
        Number of redeemable pairs = min(up_qty, down_qty)
        
        A "pair" is one UP + one DOWN token. At resolution:
        - Pair pays out $1.00 regardless of outcome
        - Your profit = $1.00 - combined_avg per pair
        
        Example:
            100 UP + 80 DOWN = 80 pairs (20 UP unmatched)
        """
        return min(self.up_qty, self.down_qty)
    
    @property
    def potential_profit(self) -> float:
        """Profit per pair if redeemed = 1.00 - combined_avg"""
        return 1.0 - self.combined_avg
    
    def summary(self):
        """Print a human-readable summary of the inventory"""
        print(f"Position: UP {self.up_qty:.0f} @ {self.up_avg:.2f} | DOWN {self.down_qty:.0f} @ {self.down_avg:.2f}")
        print(f"Combined avg: {self.combined_avg:.2f} ({int(self.combined_avg*100)}c)")
        
        imb_label = 'overweight UP' if self.imbalance > 0.05 else 'overweight DOWN' if self.imbalance < -0.05 else 'balanced'
        print(f"Imbalance: {self.imbalance:+.1%} ({imb_label})")
        print(f"Redeemable pairs: {self.pairs:.0f}")
        
        if self.potential_profit > 0:
            print(f"Potential profit: {self.potential_profit:.2f} ({int(self.potential_profit*100)}c) per pair = ${self.pairs * self.potential_profit:.2f} total")
        else:
            print(f"UNDERWATER by {-self.potential_profit:.2f} ({int(-self.potential_profit*100)}c) per pair")


# ==============================================================================
# DATA STRUCTURE: MARKET
# ==============================================================================
# Current orderbook state from Polymarket
#
# In a binary market, the asks should roughly sum to > $1.00 (overround)
# and bids should roughly sum to < $1.00 (underround)
# This spread is where market makers extract profit
# ==============================================================================

@dataclass
class Market:
    """
    Current market state from Polymarket orderbook.
    
    Attributes:
        best_ask_up: Cheapest price to BUY UP (we bid below this)
        best_bid_up: Best price someone will PAY for UP
        best_ask_down: Cheapest price to BUY DOWN (we bid below this)
        best_bid_down: Best price someone will PAY for DOWN
    
    Spread exists because:
        - Asks sum to > $1.00 (you pay premium to buy both sides)
        - Bids sum to < $1.00 (you get discount to sell both sides)
    
    Example:
        UP:   bid=50c / ask=52c  (2c spread)
        DOWN: bid=48c / ask=50c  (2c spread)
        Ask total = 52c + 50c = 102c (2% overround)
        Bid total = 50c + 48c = 98c  (2% underround)
    """
    best_ask_up: float      # Cheapest UP available (we want to bid below this)
    best_bid_up: float      # Best bid for UP
    best_ask_down: float    # Cheapest DOWN available
    best_bid_down: float    # Best bid for DOWN
    
    def summary(self):
        print(f"UP:   bid={self.best_bid_up:.2f} / ask={self.best_ask_up:.2f}")
        print(f"DOWN: bid={self.best_bid_down:.2f} / ask={self.best_ask_down:.2f}")
        print(f"Market implied: UP={self.best_ask_up:.0%}, DOWN={self.best_ask_down:.0%}")


# ==============================================================================
# DATA STRUCTURE: ORACLE
# ==============================================================================
# External price feed (e.g., BTC price from Binance/Coinbase)
#
# This is our "information edge" - we use the oracle to estimate
# the TRUE probability of UP winning before the market fully reflects it
#
# Example: Market question is "BTC > $97,000 at 2:00 PM?"
#   - If BTC is at $97,500 (0.5% above), UP is more likely to win
#   - If BTC is at $96,500 (0.5% below), DOWN is more likely to win
# ==============================================================================

@dataclass 
class Oracle:
    """
    External price oracle (e.g., BTC price from exchange WebSocket).
    
    Attributes:
        current_price: Current price from exchange (e.g., 97200)
        threshold: The market question threshold (e.g., 97000)
    
    The oracle gives us an information edge:
        - If price > threshold: UP more likely to win
        - If price < threshold: DOWN more likely to win
        - How far above/below indicates confidence level
    """
    current_price: float    # e.g., 97200 (current BTC price)
    threshold: float        # e.g., 97000 (the market question)
    
    @property
    def distance_pct(self) -> float:
        """
        How far is current price from threshold, as a percentage.
        
        Formula: (current - threshold) / threshold
        
        Examples:
            BTC=97500, threshold=97000 → +0.52% (above threshold)
            BTC=96500, threshold=97000 → -0.52% (below threshold)
            BTC=97000, threshold=97000 →  0.00% (exactly at threshold)
        
        This percentage drives our fair value estimate:
            - Large positive → high probability UP wins
            - Large negative → high probability DOWN wins
            - Near zero → 50/50 toss-up
        """
        return (self.current_price - self.threshold) / self.threshold
    
    def summary(self):
        direction = "ABOVE" if self.current_price > self.threshold else "BELOW"
        print(f"Oracle: ${self.current_price:,.0f} ({direction} ${self.threshold:,.0f} by {abs(self.distance_pct):.2%})")

NameError: name 'dataclass' is not defined

---

## 2. The Quoter Implementation

The quoter is the brain of the market maker. It takes:
- Current inventory (what we hold)
- Current market (what prices exist)
- Oracle data (external price feed)
- Time to resolution (how much time left)

And outputs:
- Bid prices for UP and DOWN (or decision to skip)
- Order sizes for each side
- Diagnostic information explaining the decision

### The 4-Layer Framework

| Layer | Purpose | Key Formula | Affects |
|-------|---------|-------------|---------|
| **1. Oracle-Adjusted Offset** | React to oracle direction | `up_offset = spread - oracle_adj` | Prices |
| **2. Adverse Selection** | Widen spread near resolution | `spread = base × (1 + 3 × P(informed))` | Base spread |
| **3. Inventory Skew** | Balance positions | `offset × (1 + γ × q)`, `size × exp(-λ × q)` | Prices AND Sizes |
| **4. Edge Check** | Don't overpay | `Skip if edge < threshold` | Quote/Skip decision |

### Key Insight: Two Things Affect Prices

1. **Oracle** (Layer 1): Tighten offset on favored side, widen on unfavored side
2. **Imbalance** (Layer 3): Widen offset on overweight side, tighten on needed side

### Key Insight: Only Imbalance Affects Sizes

- Smaller orders on overweight side
- Bigger orders on needed side

In [ ]:
# ==============================================================================
# QUOTE RESULT - OUTPUT DATA STRUCTURE
# ==============================================================================

@dataclass
class QuoteResult:
    """
    Output from the quoter - contains quotes and ALL diagnostic information.
    
    Tracks intermediate calculations from each layer for debugging.
    If bid_up or bid_down is None, we're NOT quoting that side.
    """
    # ══════════════════════════════════════════════════════════════════════════
    # FINAL QUOTES
    # ══════════════════════════════════════════════════════════════════════════
    bid_up: Optional[float]      # Final UP bid (None = skip)
    size_up: float               # Final UP size
    bid_down: Optional[float]    # Final DOWN bid (None = skip)
    size_down: float             # Final DOWN size
    
    # ══════════════════════════════════════════════════════════════════════════
    # LAYER 1: ORACLE-ADJUSTED OFFSET
    # ══════════════════════════════════════════════════════════════════════════
    oracle_adj: float            # Oracle adjustment: distance_pct × sensitivity
    raw_up_offset: float         # UP offset BEFORE inventory skew
    raw_down_offset: float       # DOWN offset BEFORE inventory skew
    
    # ══════════════════════════════════════════════════════════════════════════
    # LAYER 2: ADVERSE SELECTION
    # ══════════════════════════════════════════════════════════════════════════
    p_informed: float            # Probability of informed trade
    base_spread: float           # Base spread (includes adverse selection)
    
    # ══════════════════════════════════════════════════════════════════════════
    # LAYER 3: INVENTORY SKEW
    # ══════════════════════════════════════════════════════════════════════════
    inventory_q: float           # Imbalance: (UP - DOWN) / (UP + DOWN)
    spread_mult_up: float        # Offset multiplier for UP (>1 if overweight UP)
    spread_mult_down: float      # Offset multiplier for DOWN (<1 if overweight UP)
    final_up_offset: float       # UP offset AFTER inventory skew
    final_down_offset: float     # DOWN offset AFTER inventory skew
    raw_size_up: float           # UP size from skew formula
    raw_size_down: float         # DOWN size from skew formula
    
    # ══════════════════════════════════════════════════════════════════════════
    # LAYER 4: EDGE CHECK
    # ══════════════════════════════════════════════════════════════════════════
    edge_up: float               # Edge vs market: ask - bid
    edge_down: float
    skip_reason_up: Optional[str] = None
    skip_reason_down: Optional[str] = None


# ==============================================================================
# THE QUOTER - MAIN IMPLEMENTATION
# ==============================================================================

class InventoryMMQuoter:
    """
    Inventory Market Maker for Polymarket 15-minute binary markets.
    
    4-Layer Framework:
    1. Oracle-Adjusted Offset: React to oracle direction
    2. Adverse Selection: Widen spread near resolution  
    3. Inventory Skew: Balance positions (affects BOTH prices and sizes)
    4. Edge Check: Don't overpay
    """
    
    def __init__(
        self,
        # ══════════════════════════════════════════════════════════════════════
        # LAYER 1: Oracle-Adjusted Offset
        # ══════════════════════════════════════════════════════════════════════
        oracle_sensitivity: float = 5.0,
        # Formula: oracle_adj = oracle_distance_pct × sensitivity
        # Higher = more aggressive directional adjustment
        
        # ══════════════════════════════════════════════════════════════════════
        # LAYER 2: Adverse Selection (Glosten-Milgrom)
        # ══════════════════════════════════════════════════════════════════════
        base_spread: float = 0.02,
        # Base offset from best_ask (2c) before oracle adjustment
        
        p_informed_base: float = 0.2,
        # Base probability of informed trader (20%)
        
        time_decay_minutes: float = 5.0,
        # Time constant: p_informed = base × exp(-minutes / decay)
        
        # ══════════════════════════════════════════════════════════════════════
        # LAYER 3: Inventory Skew
        # ══════════════════════════════════════════════════════════════════════
        gamma_inv: float = 0.5,
        # Offset multiplier sensitivity: mult = 1 + gamma × imbalance
        
        lambda_size: float = 1.0,
        # Size sensitivity: size = base × exp(-lambda × imbalance)
        
        base_size: float = 50.0,
        # Base order size when balanced
        
        # ══════════════════════════════════════════════════════════════════════
        # LAYER 4: Edge Check
        # ══════════════════════════════════════════════════════════════════════
        edge_threshold: float = 0.01,
        # Minimum edge (1c) to quote
        
        min_offset: float = 0.01,
        # Minimum offset from best_ask (1c)
    ):
        self.oracle_sensitivity = oracle_sensitivity
        self.base_spread = base_spread
        self.p_informed_base = p_informed_base
        self.time_decay_minutes = time_decay_minutes
        self.gamma_inv = gamma_inv
        self.lambda_size = lambda_size
        self.base_size = base_size
        self.edge_threshold = edge_threshold
        self.min_offset = min_offset
    
    def params_summary(self) -> str:
        """Return a formatted string of all parameters."""
        return (
            f"oracle_sensitivity={self.oracle_sensitivity}, "
            f"base_spread={self.base_spread}, "
            f"p_informed_base={self.p_informed_base}, "
            f"time_decay={self.time_decay_minutes}min, "
            f"gamma={self.gamma_inv}, "
            f"lambda={self.lambda_size}, "
            f"base_size={self.base_size}, "
            f"edge_threshold={self.edge_threshold}, "
            f"min_offset={self.min_offset}"
        )
    
    # ==========================================================================
    # LAYER 1: ORACLE-ADJUSTED OFFSET
    # ==========================================================================
    def calculate_oracle_adjusted_offsets(self, oracle: Oracle, base_offset: float) -> tuple[float, float, float]:
        """
        Adjust offset from best_ask based on oracle direction.
        
        When oracle favors UP (above threshold):
          - UP offset DECREASES (tighter bid, more aggressive)
          - DOWN offset INCREASES (wider bid, protection from dumps)
        
        Returns: (oracle_adj, up_offset, down_offset)
        """
        oracle_adj = oracle.distance_pct * self.oracle_sensitivity
        up_offset = max(self.min_offset, base_offset - oracle_adj)
        down_offset = max(self.min_offset, base_offset + oracle_adj)
        return oracle_adj, up_offset, down_offset
    
    # ==========================================================================
    # LAYER 2: ADVERSE SELECTION SPREAD
    # ==========================================================================
    def calculate_adverse_selection(self, minutes_to_resolution: float) -> tuple[float, float]:
        """
        Widen spread near resolution when informed traders dominate.
        
        Formula: p_informed = base × exp(-minutes / decay)
                 spread = base_spread × (1 + 3 × p_informed)
        
        Returns: (p_informed, spread)
        """
        p_informed = self.p_informed_base * math.exp(-minutes_to_resolution / self.time_decay_minutes)
        p_informed = min(0.8, p_informed)  # Cap at 80%
        spread = self.base_spread * (1 + 3 * p_informed)
        return p_informed, spread
    
    # ==========================================================================
    # LAYER 3: INVENTORY SKEW
    # ==========================================================================
    def calculate_inventory_skew(self, inventory: Inventory) -> tuple[float, float, float, float]:
        """
        Adjust offsets and sizes based on inventory imbalance.
        
        Offset multiplier: mult = 1 + gamma × imbalance
        Size: size = base × exp(-lambda × imbalance)
        
        Returns: (spread_mult_up, spread_mult_down, size_up, size_down)
        """
        q = inventory.imbalance
        
        # SPREAD MULTIPLIER (affects final offset)
        spread_mult_up = 1 + self.gamma_inv * q      # >1 when overweight UP
        spread_mult_down = 1 - self.gamma_inv * q    # <1 when overweight UP
        
        # SIZE (affects order quantity)
        size_up = self.base_size * math.exp(-self.lambda_size * q)    # Smaller when overweight UP
        size_down = self.base_size * math.exp(self.lambda_size * q)   # Bigger when overweight UP
        
        return spread_mult_up, spread_mult_down, size_up, size_down
    
    # ==========================================================================
    # LAYER 4: EDGE CHECK
    # ==========================================================================
    def check_edge(self, bid: float, market_ask: float) -> tuple[bool, float, Optional[str]]:
        """
        Check if we have sufficient edge vs market.
        
        Returns: (should_quote, edge, skip_reason)
        """
        edge = market_ask - bid
        if edge < self.edge_threshold:
            return False, edge, f"edge {edge:.3f} < threshold {self.edge_threshold}"
        return True, edge, None
    
    # ==========================================================================
    # MAIN QUOTE GENERATION
    # ==========================================================================
    def quote(
        self,
        inventory: Inventory,
        market: Market,
        oracle: Oracle,
        minutes_to_resolution: float,
    ) -> QuoteResult:
        """
        Generate quotes using the 4-layer framework.
        
        Returns QuoteResult with all intermediate calculations for debugging.
        """
        # LAYER 2: Adverse selection (base spread)
        p_informed, base_spread = self.calculate_adverse_selection(minutes_to_resolution)
        
        # LAYER 1: Oracle-adjusted offsets
        oracle_adj, raw_up_offset, raw_down_offset = self.calculate_oracle_adjusted_offsets(oracle, base_spread)
        
        # LAYER 3: Inventory skew
        spread_mult_up, spread_mult_down, raw_size_up, raw_size_down = self.calculate_inventory_skew(inventory)
        
        # Apply inventory skew to offsets
        final_up_offset = raw_up_offset * spread_mult_up
        final_down_offset = raw_down_offset * spread_mult_down
        
        # Calculate bids
        raw_bid_up = market.best_bid_up - final_up_offset
        raw_bid_down = market.best_bid_down - final_down_offset
        
        # Snap to tick
        bid_up = snap_to_tick(raw_bid_up)
        bid_down = snap_to_tick(raw_bid_down)
        
        # LAYER 4: Edge check
        quote_up, edge_up, skip_up = self.check_edge(bid_up, market.best_ask_up)
        quote_down, edge_down, skip_down = self.check_edge(bid_down, market.best_ask_down)
        
        return QuoteResult(
            # Final quotes
            bid_up=bid_up if quote_up else None,
            size_up=round(raw_size_up) if quote_up else 0,
            bid_down=bid_down if quote_down else None,
            size_down=round(raw_size_down) if quote_down else 0,
            # Layer 1
            oracle_adj=oracle_adj,
            raw_up_offset=raw_up_offset,
            raw_down_offset=raw_down_offset,
            # Layer 2
            p_informed=p_informed,
            base_spread=base_spread,
            # Layer 3
            inventory_q=inventory.imbalance,
            spread_mult_up=spread_mult_up,
            spread_mult_down=spread_mult_down,
            final_up_offset=final_up_offset,
            final_down_offset=final_down_offset,
            raw_size_up=raw_size_up,
            raw_size_down=raw_size_down,
            # Layer 4
            edge_up=edge_up,
            edge_down=edge_down,
            skip_reason_up=skip_up,
            skip_reason_down=skip_down,
        )

---
## 3. Pretty Print Results

In [ ]:
def print_quote_result(result: QuoteResult, inventory: Inventory, market: Market, oracle: Oracle, minutes: float, quoter: InventoryMMQuoter = None):
    """
    Pretty print the quote result with FULL layer-by-layer diagnostics.
    
    Shows all intermediate calculations so you can trace exactly how each
    layer contributes to the final quote.
    """
    print("=" * 80)
    print("                    INVENTORY MM QUOTE OUTPUT")
    print("=" * 80)
    
    # ══════════════════════════════════════════════════════════════════════════
    # INPUTS
    # ══════════════════════════════════════════════════════════════════════════
    print("\n┌" + "─" * 78 + "┐")
    print("│ INPUTS" + " " * 71 + "│")
    print("├" + "─" * 78 + "┤")
    
    # Oracle
    direction = "ABOVE" if oracle.current_price > oracle.threshold else "BELOW" if oracle.current_price < oracle.threshold else "AT"
    print(f"│ Oracle:    ${oracle.current_price:,.0f} ({direction} ${oracle.threshold:,.0f} by {abs(oracle.distance_pct):.2%})" + " " * 20 + "│")
    
    # Market
    print(f"│ Market:    UP ask={market.best_ask_up:.2f} bid={market.best_bid_up:.2f}  |  DOWN ask={market.best_ask_down:.2f} bid={market.best_bid_down:.2f}" + " " * 10 + "│")
    
    # Inventory
    imb_label = 'overweight UP' if inventory.imbalance > 0.05 else 'overweight DOWN' if inventory.imbalance < -0.05 else 'balanced'
    print(f"│ Inventory: UP {inventory.up_qty:.0f}@{inventory.up_avg:.2f}  DOWN {inventory.down_qty:.0f}@{inventory.down_avg:.2f}  (q={inventory.imbalance:+.1%} {imb_label})" + " " * 5 + "│")
    print(f"│ Combined:  {inventory.combined_avg:.2f} ({int(inventory.combined_avg*100)}c) " + ("PROFITABLE" if inventory.combined_avg < 1.0 else "UNDERWATER" if inventory.combined_avg > 1.0 else "BREAKEVEN") + " " * 30 + "│")
    print(f"│ Time:      {minutes:.1f} minutes to resolution" + " " * 40 + "│")
    print("└" + "─" * 78 + "┘")
    
    # ══════════════════════════════════════════════════════════════════════════
    # LAYER-BY-LAYER CALCULATIONS
    # ══════════════════════════════════════════════════════════════════════════
    print("\n┌" + "─" * 78 + "┐")
    print("│ LAYER-BY-LAYER CALCULATIONS" + " " * 50 + "│")
    print("├" + "─" * 78 + "┤")
    
    # Layer 2 (shown first because it provides base_spread for Layer 1)
    print("│" + " " * 78 + "│")
    print("│ LAYER 2: ADVERSE SELECTION" + " " * 51 + "│")
    print(f"│   p_informed = {result.p_informed:.1%} (probability of informed trade)" + " " * 30 + "│")
    print(f"│   base_spread = {result.base_spread:.3f} ({result.base_spread*100:.1f}c)" + " " * 45 + "│")
    
    # Layer 1
    print("│" + " " * 78 + "│")
    print("│ LAYER 1: ORACLE-ADJUSTED OFFSET" + " " * 46 + "│")
    print(f"│   oracle_adj = {result.oracle_adj:+.3f} ({result.oracle_adj*100:+.1f}c)" + " " * 45 + "│")
    
    up_tight = "TIGHT" if result.raw_up_offset < result.base_spread else ""
    down_tight = "TIGHT" if result.raw_down_offset < result.base_spread else ""
    print(f"│   UP offset (raw)   = {result.base_spread:.3f} - {result.oracle_adj:+.3f} = {result.raw_up_offset:.3f} ({result.raw_up_offset*100:.1f}c) {up_tight}" + " " * 15 + "│")
    print(f"│   DOWN offset (raw) = {result.base_spread:.3f} + {result.oracle_adj:+.3f} = {result.raw_down_offset:.3f} ({result.raw_down_offset*100:.1f}c) {down_tight}" + " " * 15 + "│")
    
    # Layer 3
    print("│" + " " * 78 + "│")
    print("│ LAYER 3: INVENTORY SKEW" + " " * 54 + "│")
    print(f"│   imbalance q = {result.inventory_q:+.1%}" + " " * 55 + "│")
    print(f"│   " + " " * 75 + "│")
    print(f"│   OFFSET MULTIPLIERS:" + " " * 56 + "│")
    print(f"│     UP mult   = 1 + {quoter.gamma_inv if quoter else 0.5:.1f} × {result.inventory_q:+.2f} = {result.spread_mult_up:.2f}" + " " * 35 + "│")
    print(f"│     DOWN mult = 1 - {quoter.gamma_inv if quoter else 0.5:.1f} × {result.inventory_q:+.2f} = {result.spread_mult_down:.2f}" + " " * 35 + "│")
    print(f"│   " + " " * 75 + "│")
    print(f"│   FINAL OFFSETS (raw × mult):" + " " * 47 + "│")
    print(f"│     UP offset   = {result.raw_up_offset:.3f} × {result.spread_mult_up:.2f} = {result.final_up_offset:.3f} ({result.final_up_offset*100:.1f}c)" + " " * 25 + "│")
    print(f"│     DOWN offset = {result.raw_down_offset:.3f} × {result.spread_mult_down:.2f} = {result.final_down_offset:.3f} ({result.final_down_offset*100:.1f}c)" + " " * 25 + "│")
    print(f"│   " + " " * 75 + "│")
    print(f"│   SIZES:" + " " * 69 + "│")
    print(f"│     UP size   = {quoter.base_size if quoter else 50:.0f} × exp(-{quoter.lambda_size if quoter else 1:.1f} × {result.inventory_q:+.2f}) = {result.raw_size_up:.1f} → {round(result.raw_size_up):.0f}" + " " * 20 + "│")
    print(f"│     DOWN size = {quoter.base_size if quoter else 50:.0f} × exp(+{quoter.lambda_size if quoter else 1:.1f} × {result.inventory_q:+.2f}) = {result.raw_size_down:.1f} → {round(result.raw_size_down):.0f}" + " " * 20 + "│")
    
    # Final bids
    print("│" + " " * 78 + "│")
    print("│ BID CALCULATION (best_ask - final_offset):" + " " * 34 + "│")
    print(f"│   UP bid   = {market.best_ask_up:.2f} - {result.final_up_offset:.3f} = {market.best_ask_up - result.final_up_offset:.3f} → {snap_to_tick(market.best_ask_up - result.final_up_offset):.2f}" + " " * 25 + "│")
    print(f"│   DOWN bid = {market.best_ask_down:.2f} - {result.final_down_offset:.3f} = {market.best_ask_down - result.final_down_offset:.3f} → {snap_to_tick(market.best_ask_down - result.final_down_offset):.2f}" + " " * 25 + "│")
    
    # Layer 4
    print("│" + " " * 78 + "│")
    print("│ LAYER 4: EDGE CHECK" + " " * 58 + "│")
    threshold = quoter.edge_threshold if quoter else 0.01
    up_pass = "PASS" if result.edge_up >= threshold else "FAIL"
    down_pass = "PASS" if result.edge_down >= threshold else "FAIL"
    print(f"│   UP edge   = {market.best_ask_up:.2f} - {result.bid_up or 0:.2f} = {result.edge_up:.3f} ({result.edge_up*100:.1f}c) >= {threshold:.2f}? {up_pass}" + " " * 15 + "│")
    print(f"│   DOWN edge = {market.best_ask_down:.2f} - {result.bid_down or 0:.2f} = {result.edge_down:.3f} ({result.edge_down*100:.1f}c) >= {threshold:.2f}? {down_pass}" + " " * 15 + "│")
    
    print("└" + "─" * 78 + "┘")
    
    # ══════════════════════════════════════════════════════════════════════════
    # FINAL OUTPUT
    # ══════════════════════════════════════════════════════════════════════════
    print("\n┌" + "─" * 78 + "┐")
    print("│ FINAL QUOTES" + " " * 65 + "│")
    print("├" + "─" * 78 + "┤")
    
    if result.bid_up is not None:
        print(f"│   UP:   BID {result.bid_up:.2f} ({int(result.bid_up*100)}c) for {result.size_up:.0f} shares   [edge: {result.edge_up*100:.0f}c]" + " " * 25 + "│")
    else:
        print(f"│   UP:   SKIP - {result.skip_reason_up}" + " " * 40 + "│")
    
    if result.bid_down is not None:
        print(f"│   DOWN: BID {result.bid_down:.2f} ({int(result.bid_down*100)}c) for {result.size_down:.0f} shares   [edge: {result.edge_down*100:.0f}c]" + " " * 25 + "│")
    else:
        print(f"│   DOWN: SKIP - {result.skip_reason_down}" + " " * 40 + "│")
    
    print("│" + " " * 78 + "│")
    
    if result.bid_up and result.bid_down:
        combined_bid = result.bid_up + result.bid_down
        profit_per_pair = 1.0 - combined_bid
        status = f"{profit_per_pair*100:.0f}c PROFIT" if profit_per_pair > 0 else "BREAKEVEN" if profit_per_pair == 0 else f"{-profit_per_pair*100:.0f}c LOSS"
        print(f"│   Combined bid: {combined_bid:.2f} ({int(combined_bid*100)}c) → {status} per pair if both fill" + " " * 15 + "│")
    else:
        print(f"│   Not quoting both sides - waiting for better prices" + " " * 25 + "│")
    
    print("└" + "─" * 78 + "┘")


def print_quote_compact(result: QuoteResult, inventory: Inventory, market: Market, oracle: Oracle, minutes: float):
    """Compact one-line summary for comparison tables."""
    oracle_str = f"{oracle.distance_pct*100:+.2f}%"
    q_str = f"{result.inventory_q:+.0%}"
    
    bid_up = f"{int(result.bid_up*100)}c" if result.bid_up else "SKIP"
    bid_down = f"{int(result.bid_down*100)}c" if result.bid_down else "SKIP"
    
    combined = f"{int((result.bid_up + result.bid_down)*100)}c" if result.bid_up and result.bid_down else "N/A"
    
    print(f"Oracle:{oracle_str:>7} | q:{q_str:>5} | Bids: UP={bid_up:>4} DOWN={bid_down:>4} | Sizes: {result.size_up:.0f}/{result.size_down:.0f} | Combined: {combined}")

---
## 4. Test Scenarios

### Understanding Adverse Selection (Layer 2)

**The Problem:** Near resolution, some traders become "informed" - they can see which way the market is going. They dump the losing side on market makers like you.

**Timeline of Risk:**

```
14 min ────────── 7 min ────────── 3 min ────────── 1 min ────────── 0 (Resolution)
   │                │                │                │                │
 Spread: 2.1c     2.3c             2.7c             3.0c          STOP QUOTING
 P(inf): 1%       5%               11%              16%             100%
   │                │                │                │                │
   └── Safe ───────┴── Careful ────┴── Cautious ───┴── DANGER ──────┘
```

**Why Wider Spreads Help:**

- At 14 min: Low risk, tight spreads (2c offset) → more fills
- At 1 min: High risk, wider spreads (3c offset) → pay less if dumped on
- If dumped at 46c instead of 48c, you save 2c × 100 tokens = $2

**The Formula:**

```python
p_informed = p_informed_base × exp(-minutes / time_decay)
spread = base_spread × (1 + 3 × p_informed)
```

This automatically widens your offset as resolution approaches.

In [ ]:
# ==============================================================================
# QUOTER INITIALIZATION
# ==============================================================================

# Create quoter with default parameters
quoter = InventoryMMQuoter(
    # Layer 1: Oracle
    oracle_sensitivity=5.0,      # How much oracle shifts offset
    
    # Layer 2: Adverse Selection
    base_spread=0.01,            # 2c base offset
    p_informed_base=0.2,         # 20% base informed probability
    time_decay_minutes=5.0,      # Time constant for adverse selection
    
    # Layer 3: Inventory Skew  
    gamma_inv=1.5,               # Offset multiplier sensitivity
    lambda_size=1.5,             # Size sensitivity
    base_size=50.0,              # Base order size
    
    # Layer 4: Edge Check
    edge_threshold=0.01,         # 1c minimum edge
    min_offset=0.01,             # 1c minimum offset
)

print("=" * 60)
print("QUOTER INITIALIZED")
print("=" * 60)
print()
print("Layer 1 - Oracle Adjustment:")
print(f"  oracle_sensitivity = {quoter.oracle_sensitivity}")
print()
print("Layer 2 - Adverse Selection:")
print(f"  base_spread        = {quoter.base_spread} ({quoter.base_spread*100:.0f}c)")
print(f"  p_informed_base    = {quoter.p_informed_base} ({quoter.p_informed_base*100:.0f}%)")
print(f"  time_decay_minutes = {quoter.time_decay_minutes}")
print()
print("Layer 3 - Inventory Skew:")
print(f"  gamma_inv          = {quoter.gamma_inv} (offset mult sensitivity)")
print(f"  lambda_size        = {quoter.lambda_size} (size sensitivity)")
print(f"  base_size          = {quoter.base_size}")
print()
print("Layer 4 - Edge Check:")
print(f"  edge_threshold     = {quoter.edge_threshold} ({quoter.edge_threshold*100:.0f}c)")
print(f"  min_offset         = {quoter.min_offset} ({quoter.min_offset*100:.0f}c)")
print()
print("=" * 60)


# ==============================================================================
# HELPER FUNCTION: CREATE REALISTIC COMPLEMENTARY MARKET
# ==============================================================================

def create_market(up_mid: float, spread: float = 0.02) -> Market:
    """
    Create a realistic COMPLEMENTARY orderbook.
    
    In Polymarket binary markets, UP + DOWN = $1.00 at resolution.
    Therefore orderbooks are complementary:
        UP_ask ≈ 1 - DOWN_bid
        DOWN_ask ≈ 1 - UP_bid
    
    This function ensures asks sum to approximately $1.00 + overround.
    
    Args:
        up_mid: Midpoint for UP probability (e.g., 0.55 means UP is 55% likely)
        spread: Bid-ask spread on each side (default 2c)
    
    Returns:
        Market with complementary prices
    
    Example:
        create_market(up_mid=0.55, spread=0.02)
        → UP: bid=0.54, ask=0.56
        → DOWN: bid=0.44, ask=0.46
        → Asks sum to 1.02 (2% overround)
        → Combined bid guaranteed < 1.00!
    """
    down_mid = 1.0 - up_mid
    return Market(
        best_ask_up=round(up_mid + spread/2, 2),
        best_bid_up=round(up_mid - spread/2, 2),
        best_ask_down=round(down_mid + spread/2, 2),
        best_bid_down=round(down_mid - spread/2, 2),
    )


# ==============================================================================
# HELPER FUNCTION: RUN SCENARIO
# ==============================================================================

def run_scenario(
    name: str,
    # Inventory
    up_qty: float, up_avg: float,
    down_qty: float, down_avg: float,
    # Market (use create_market() for realistic data!)
    up_ask: float, up_bid: float,
    down_ask: float, down_bid: float,
    # Oracle
    oracle_price: float, threshold: float,
    # Time
    minutes: float,
    # Options
    verbose: bool = True,
    quoter_instance: InventoryMMQuoter = None,
) -> QuoteResult:
    """
    Helper to quickly run a scenario and print results.
    
    TIP: Use create_market() to generate realistic complementary market data:
        mkt = create_market(up_mid=0.55)  # 55% UP probability
        run_scenario(..., up_ask=mkt.best_ask_up, ...)
    
    Example:
        run_scenario(
            "My Test",
            up_qty=100, up_avg=0.50,
            down_qty=80, down_avg=0.48,
            up_ask=0.56, up_bid=0.54,    # From create_market(0.55)
            down_ask=0.46, down_bid=0.44,
            oracle_price=97200, threshold=97000,
            minutes=8.0,
        )
    """
    q = quoter_instance or quoter
    
    inv = Inventory(up_qty, up_avg, down_qty, down_avg)
    mkt = Market(up_ask, up_bid, down_ask, down_bid)
    orc = Oracle(oracle_price, threshold)
    
    result = q.quote(inv, mkt, orc, minutes)
    
    if verbose:
        print(f"\n{'='*80}")
        print(f"SCENARIO: {name}")
        print(f"{'='*80}")
        print_quote_result(result, inv, mkt, orc, minutes, q)
    
    return result


print("\nHelper functions available:")
print("  - create_market(up_mid, spread) → Realistic complementary orderbook")
print("  - run_scenario(...) → Run and print a test scenario")
print()
print("Example: create_market(0.55) gives UP ask=0.56, DOWN ask=0.46 (sum=1.02)")

### Scenario 1: Balanced Position, Oracle Neutral (10 min left)

**Setup:** Equal inventory, oracle at threshold, plenty of time.
**Expected:** Symmetric quotes, modest spread.

In [ ]:
# Scenario 1: Balanced position, oracle neutral, plenty of time
# Market: 50/50 probability, 2c spread each side → asks sum to 1.02
run_scenario(
    "Balanced, Oracle Neutral, 10 min",
    up_qty=100, up_avg=0.48,
    down_qty=100, down_avg=0.48,
    up_ask=0.51, up_bid=0.49,     # 50% UP mid
    down_ask=0.51, down_bid=0.49, # 50% DOWN mid
    oracle_price=97000, threshold=97000,  # Exactly at threshold
    minutes=10.0,
)

### Scenario 2: Overweight UP, Oracle Says UP Likely (7 min left)

**Setup:** Heavy UP position, oracle confirms UP direction, underwater.
**Expected:** Tight UP offset (oracle favors), wide DOWN offset, smaller UP size, bigger DOWN size.

In [ ]:
# Scenario 2: Overweight UP, oracle says UP likely, underwater
# Market: 65/35 probability (UP favored), asks sum to 1.02
run_scenario(
    "Overweight UP, Oracle Bullish, 7 min",
    up_qty=150, up_avg=0.55,
    down_qty=50, down_avg=0.50,
    up_ask=0.66, up_bid=0.64,     # 65% UP mid
    down_ask=0.36, down_bid=0.34, # 35% DOWN mid (complementary!)
    oracle_price=97500, threshold=97000,  # 0.5% above threshold
    minutes=7.0,
)

### Scenario 3: Wrong-Footed (Long UP but Oracle Says DOWN, 3 min left)

**Setup:** Heavy UP position but oracle moved against us, little time left.
**Expected:** Wide UP offset (oracle + imbalance both penalize), tight DOWN offset, high adverse selection.

In [ ]:
# Scenario 3: DANGER - Long UP but oracle moved against us
# Market: 35/65 probability (DOWN favored), asks sum to 1.02
run_scenario(
    "Wrong-Footed (Long UP, Oracle Bearish), 3 min",
    up_qty=150, up_avg=0.60,
    down_qty=50, down_avg=0.40,
    up_ask=0.36, up_bid=0.34,     # 35% UP mid (DOWN winning!)
    down_ask=0.66, down_bid=0.64, # 65% DOWN mid (complementary!)
    oracle_price=96500, threshold=97000,  # 0.5% BELOW threshold!
    minutes=3.0,
)

### Scenario 4: Near Resolution - High Adverse Selection (1 min left)

**Setup:** Balanced position, oracle slightly bullish, very close to resolution.
**Expected:** Wide spreads on both sides due to high P(informed), reduced fill likelihood.

In [ ]:
# Scenario 4: Very close to resolution - high adverse selection
# Market: 55/45 probability (slightly UP), asks sum to 1.02
run_scenario(
    "Near Resolution (High Adverse Selection), 1 min",
    up_qty=100, up_avg=0.50,
    down_qty=100, down_avg=0.48,
    up_ask=0.56, up_bid=0.54,     # 55% UP mid
    down_ask=0.46, down_bid=0.44, # 45% DOWN mid (complementary!)
    oracle_price=97100, threshold=97000,  # Slightly above
    minutes=1.0,
)

### Scenario 5: Deeply Underwater (8 min left)

**Setup:** Balanced position but combined avg > $1.00 (underwater).
**Expected:** Normal quotes - balance mechanism doesn't care about historical avg, only current market.

In [ ]:
# Scenario 5: Deeply underwater - balance doesn't care about historical avg
# Market: 50/50 probability, asks sum to 1.02
run_scenario(
    "Deeply Underwater (13c loss), 8 min",
    up_qty=100, up_avg=0.58,
    down_qty=100, down_avg=0.55,
    up_ask=0.51, up_bid=0.49,     # 50% UP mid
    down_ask=0.51, down_bid=0.49, # 50% DOWN mid (complementary!)
    oracle_price=97000, threshold=97000,  # Neutral
    minutes=8.0,
)

---
## 5. Scenario Comparison Table

In [ ]:
# ==============================================================================
# SCENARIO COMPARISON TABLE
# ==============================================================================
# All scenarios now use REALISTIC COMPLEMENTARY orderbooks
# UP_ask + DOWN_ask ≈ 1.02 (2% overround) in all cases
# Combined bid is ALWAYS < $1.00 by construction!

scenarios = [
    # (name, up_qty, up_avg, down_qty, down_avg, up_ask, up_bid, down_ask, down_bid, oracle, threshold, minutes)
    # Market data now uses complementary prices!
    
    # 50/50 markets (UP mid = 0.50)
    ("Balanced, Neutral, 10min",      100, 0.48, 100, 0.48, 0.51, 0.49, 0.51, 0.49, 97000, 97000, 10.0),
    ("Breakeven position, 12min",     100, 0.50, 100, 0.50, 0.51, 0.49, 0.51, 0.49, 97000, 97000, 12.0),
    ("Profitable 4c, 9min",           100, 0.48, 100, 0.48, 0.51, 0.49, 0.51, 0.49, 97000, 97000, 9.0),
    ("Underwater 13c, 8min",          100, 0.58, 100, 0.55, 0.51, 0.49, 0.51, 0.49, 97000, 97000, 8.0),
    ("Fresh start, 14min",              0, 0.50,   0, 0.50, 0.51, 0.49, 0.51, 0.49, 97000, 97000, 14.0),
    
    # UP-favored markets (UP mid = 0.55-0.65)
    ("Balanced, UP likely 55%, 10min", 100, 0.48, 100, 0.48, 0.56, 0.54, 0.46, 0.44, 97300, 97000, 10.0),
    ("Overweight UP, UP likely, 7min", 150, 0.55,  50, 0.50, 0.66, 0.64, 0.36, 0.34, 97500, 97000, 7.0),
    
    # DOWN-favored markets (UP mid = 0.35-0.45)
    ("Overweight UP, DOWN likely, 3m", 150, 0.60,  50, 0.40, 0.36, 0.34, 0.66, 0.64, 96500, 97000, 3.0),
    ("Heavy DOWN imbalance, 5min",     30, 0.45, 120, 0.55, 0.46, 0.44, 0.56, 0.54, 96800, 97000, 5.0),
    
    # Near resolution (high adverse selection)
    ("Near resolution, 1min",         100, 0.50, 100, 0.48, 0.56, 0.54, 0.46, 0.44, 97100, 97000, 1.0),
    
    # Extreme prices (UP mid = 0.80 or 0.20)
    ("High price UP 80c, 6min",        80, 0.75,  40, 0.22, 0.81, 0.79, 0.21, 0.19, 97800, 97000, 6.0),
    ("High price DOWN 85c, 6min",      40, 0.12,  80, 0.80, 0.16, 0.14, 0.86, 0.84, 96200, 97000, 6.0),
]

print("=" * 140)
print("SCENARIO COMPARISON TABLE (Realistic Complementary Orderbooks)")
print("=" * 140)
print()
print(f"{'Scenario':<30} {'q':>6} {'oracle':>8} │ {'L1 offset':>14} {'L3 mult':>12} {'final off':>14} │ {'bid':>12} {'size':>10} │ {'comb':>6}")
print("─" * 140)

for name, up_qty, up_avg, down_qty, down_avg, up_ask, up_bid, down_ask, down_bid, orc_price, threshold, mins in scenarios:
    inv = Inventory(up_qty, up_avg, down_qty, down_avg)
    mkt = Market(up_ask, up_bid, down_ask, down_bid)
    orc = Oracle(orc_price, threshold)
    r = quoter.quote(inv, mkt, orc, mins)
    
    # Format columns
    q_str = f"{r.inventory_q:+.0%}"
    oracle_str = f"{r.oracle_adj*100:+.1f}c"
    
    # Layer 1: raw offsets
    l1_str = f"{r.raw_up_offset*100:.1f}/{r.raw_down_offset*100:.1f}c"
    
    # Layer 3: multipliers
    l3_str = f"{r.spread_mult_up:.2f}/{r.spread_mult_down:.2f}"
    
    # Final offsets
    final_str = f"{r.final_up_offset*100:.1f}/{r.final_down_offset*100:.1f}c"
    
    # Bids
    bid_up = f"{int(r.bid_up*100)}c" if r.bid_up else "SKIP"
    bid_dn = f"{int(r.bid_down*100)}c" if r.bid_down else "SKIP"
    bid_str = f"{bid_up}/{bid_dn}"
    
    # Sizes
    size_str = f"{r.size_up:.0f}/{r.size_down:.0f}"
    
    # Combined - should ALWAYS be < 100c now!
    if r.bid_up and r.bid_down:
        comb_str = f"{int((r.bid_up + r.bid_down)*100)}c"
    else:
        comb_str = "N/A"
    
    print(f"{name:<30} {q_str:>6} {oracle_str:>8} │ {l1_str:>14} {l3_str:>12} {final_str:>14} │ {bid_str:>12} {size_str:>10} │ {comb_str:>6}")

print("=" * 140)
print()
print("KEY INSIGHT: Combined bid is ALWAYS < $1.00!")
print("This is GUARANTEED by the complementary orderbook structure:")
print("  - UP_ask + DOWN_ask ≈ $1.02 (2% overround)")
print("  - We bid 2-4c BELOW each ask")
print("  - Combined bid = UP_bid + DOWN_bid < UP_ask + DOWN_ask")
print("  - Therefore: Combined bid < $1.02 - 4c = $0.98 (PROFIT!)")
print()
print("Column Legend:")
print("  q = imbalance, oracle = L1 adjustment, L1 offset = raw offsets")
print("  L3 mult = inventory skew multipliers, final = offsets after skew")
print("  bid = final prices, size = order sizes, comb = combined bid")

---
## 6. Interactive Testing

In [ ]:
# ==============================================================================
# INTERACTIVE TESTING - MODIFY THESE VALUES
# ==============================================================================

# TIP: Use create_market() for realistic complementary prices!
# Example: mkt = create_market(up_mid=0.55) gives UP ask=0.56, DOWN ask=0.46

# Let's use create_market to ensure realistic data:
mkt = create_market(up_mid=0.55, spread=0.02)  # 55% UP probability, 2c spread
print("Created market with create_market(up_mid=0.55, spread=0.02):")
print(f"  UP:   bid={mkt.best_bid_up:.2f} / ask={mkt.best_ask_up:.2f}")
print(f"  DOWN: bid={mkt.best_bid_down:.2f} / ask={mkt.best_ask_down:.2f}")
print(f"  Asks sum to: {mkt.best_ask_up + mkt.best_ask_down:.2f} (should be ~1.02)")
print()

run_scenario(
    "My Custom Scenario",
    
    # Your inventory
    up_qty=100,      up_avg=0.52,
    down_qty=80,     down_avg=0.48,
    
    # Market from create_market (complementary prices!)
    up_ask=mkt.best_ask_up,     up_bid=mkt.best_bid_up,
    down_ask=mkt.best_ask_down, down_bid=mkt.best_bid_down,
    
    # Oracle (from your WebSocket)
    oracle_price=97200,   # Current BTC price
    threshold=97000,      # Market question threshold
    
    # Time
    minutes=8.0,          # Minutes to resolution (0-15)
)

---
## 7. Parameter Sensitivity

In [ ]:
# ==============================================================================
# PARAMETER SENSITIVITY
# ==============================================================================
# Compare how different parameter settings affect quotes for the same scenario
# Using REALISTIC complementary market data

# Test scenario: Slightly overweight UP, oracle bullish, 7 min left
test_inv = Inventory(120, 0.52, 80, 0.48)
test_mkt = create_market(up_mid=0.55, spread=0.02)  # Complementary: asks sum to 1.02
test_orc = Oracle(97200, 97000)
test_mins = 7.0

print(f"Test Market: UP ask={test_mkt.best_ask_up:.2f}, DOWN ask={test_mkt.best_ask_down:.2f}")
print(f"Asks sum to: {test_mkt.best_ask_up + test_mkt.best_ask_down:.2f}")
print()

configs = {
    "Conservative": InventoryMMQuoter(
        base_spread=0.03,      # 3c base (wider)
        gamma_inv=0.3,         # Lower skew sensitivity
        edge_threshold=0.02,   # 2c edge required
        min_offset=0.02,       # 2c minimum offset
    ),
    "Default": InventoryMMQuoter(
        base_spread=0.02,      # 2c base
        gamma_inv=0.5,         # Medium skew sensitivity
        edge_threshold=0.01,   # 1c edge required
        min_offset=0.01,       # 1c minimum offset
    ),
    "Aggressive": InventoryMMQuoter(
        base_spread=0.01,      # 1c base (tight)
        gamma_inv=0.7,         # Higher skew sensitivity
        edge_threshold=0.005,  # 0.5c edge required
        min_offset=0.005,      # 0.5c minimum offset
    ),
}

print("=" * 120)
print("PARAMETER SENSITIVITY COMPARISON")
print("=" * 120)
print()
print(f"Test Scenario: UP 120@52c, DOWN 80@48c | Market: UP 54/56c, DOWN 44/46c | Oracle: +0.2% | 7 min")
print()
print(f"{'Config':<15} {'spread':>8} {'gamma':>6} {'edge_th':>8} │ {'L1 off':>12} {'L3 mult':>12} {'final':>12} │ {'bids':>12} {'sizes':>10} │ {'comb':>6}")
print("─" * 120)

for name, q in configs.items():
    r = q.quote(test_inv, test_mkt, test_orc, test_mins)
    
    # Layer 1
    l1_str = f"{r.raw_up_offset*100:.1f}/{r.raw_down_offset*100:.1f}c"
    
    # Layer 3
    l3_str = f"{r.spread_mult_up:.2f}/{r.spread_mult_down:.2f}"
    final_str = f"{r.final_up_offset*100:.1f}/{r.final_down_offset*100:.1f}c"
    
    # Output
    bid_up = f"{int(r.bid_up*100)}c" if r.bid_up else "SKIP"
    bid_dn = f"{int(r.bid_down*100)}c" if r.bid_down else "SKIP"
    bid_str = f"{bid_up}/{bid_dn}"
    sz_str = f"{r.size_up:.0f}/{r.size_down:.0f}"
    
    # Combined - should ALWAYS be < 100c now!
    if r.bid_up and r.bid_down:
        comb_str = f"{int((r.bid_up+r.bid_down)*100)}c"
    else:
        comb_str = "N/A"
    
    print(f"{name:<15} {q.base_spread*100:>7.0f}c {q.gamma_inv:>6.1f} {q.edge_threshold*100:>7.1f}c │ {l1_str:>12} {l3_str:>12} {final_str:>12} │ {bid_str:>12} {sz_str:>10} │ {comb_str:>6}")

print("=" * 120)
print()
print("Observations:")
print("  - Conservative: Wider spreads, more edge required → safer but fewer fills")
print("  - Default:      Balanced approach")
print("  - Aggressive:   Tighter spreads → more fills, but higher adverse selection risk")
print()
print("NOTE: Combined bid is ALWAYS < 100c because:")
print("  Asks sum to 102c, we bid 2-6c below each → combined < 98c")

---

## Summary: The 4-Layer Framework

### Layer-by-Layer Breakdown

| Layer | Input | Output | Effect |
|-------|-------|--------|--------|
| **1. Oracle Offset** | `oracle.distance_pct` | `oracle_adj` | Tighter offset on favored side |
| **2. Adverse Selection** | `minutes_to_resolution` | `base_spread`, `p_informed` | Wider spread near resolution |
| **3. Inventory Skew** | `inventory.imbalance` | `spread_mult`, `size` | Wider offset + smaller size on overweight side |
| **4. Edge Check** | `bid`, `market_ask` | `quote` or `skip` | Don't bid too close to ask |

### What Affects What

| Factor | Affects Prices? | Affects Sizes? |
|--------|-----------------|----------------|
| **Oracle** (price vs threshold) | Yes | No |
| **Imbalance** (UP vs DOWN qty) | Yes | Yes |
| **Time** (minutes to resolution) | Yes (via spread) | No |

### Key Formulas

```python
# Layer 1: Oracle-Adjusted Offset
oracle_adj = oracle_distance_pct × sensitivity
up_offset = base_spread - oracle_adj   # Tighter when UP favored
down_offset = base_spread + oracle_adj # Wider when UP favored

# Layer 2: Adverse Selection
p_informed = base × exp(-minutes_left / decay)
spread = base_spread × (1 + 3 × p_informed)

# Layer 3: Inventory Skew (PRICES)
spread_mult_up = 1 + gamma × imbalance    # >1 when overweight UP
spread_mult_down = 1 - gamma × imbalance  # <1 when overweight UP
final_offset = offset × spread_mult

# Layer 3: Inventory Skew (SIZES)
size_up = base_size × exp(-lambda × imbalance)   # Smaller when overweight UP
size_down = base_size × exp(+lambda × imbalance) # Bigger when overweight UP

# Final Bid
bid = best_ask - final_offset

# Layer 4: Edge Check
edge = market_ask - bid
quote_if = edge >= threshold
```

### The Philosophy

> **"Balance both sides, capture spread on each."**

- Always quote BOTH UP and DOWN
- Oracle adjustment reacts to market direction
- Inventory skew pushes toward balance
- Combined cost naturally tracks market ≈ $1.00
- Profit comes from spread capture

### Reference

Adapted from O'Hara, M. (1995). *Market Microstructure Theory*
- Chapter 2: Inventory Models
- Chapter 3: Information-Based Models (Glosten-Milgrom)